In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import *
from model import *
from tqdm import tqdm_notebook as tqdm
from pathlib import Path

I0707 04:52:15.475672 140470358935360 file_utils.py:39] PyTorch version 1.5.1+cu101 available.
I0707 04:52:17.135314 140470358935360 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


# Hyper Param (Train)

In [3]:
TEST = True
LOG = True

data_dir = './FGC_release_1.7.13/'
data_file = data_dir + 'FGC_release_all_dev.json'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

BATCH_SIZE = 64
EPOCHS = 6
LR = 0.00001

cuda


# Load Data using dataset.py

In [13]:
train_set = FGC_Dataset(data_file, mode="train")

In [14]:
trainloader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [6]:
# calc pos weight for BCE
total = 0
true_cnt = 0
for instance in train_set:
    if(instance[-1] == True):
        true_cnt += 1
    total += 1
print(true_cnt)
print(total)
print(torch.tensor([total/true_cnt, total/(total-true_cnt)]))
# to increase the value of recall in the model's criterion
pos_weight = print(torch.tensor([(total-true_cnt)/true_cnt, 1]))
print(pos_weight)
# no need to applied pos_weight = torch.tensor([total/true_cnt, total/(1-true_cnt)])?

547
8843
tensor([16.1664,  1.0659])
tensor([15.1664,  1.0000])
None


# Build and Train Model

In [12]:
bert_encoder = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)

I0707 04:53:38.256322 140470358935360 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0707 04:53:50.536509 140470358935360 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
I0707 04:53:50.539288 140470358935360 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_siz

In [8]:
def calc_f1(pref, target):
    TP += ((pred == 1) & (target == 1)).cpu().sum()
    TN += ((pred == 0) & (target == 0)).cpu().sum()
    FN += ((pred == 0) & (target == 1)).cpu().sum()
    FP += ((pred == 1) & (target == 0)).cpu().sum()

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * recall * precision / (recall + precision)
    acc = (TP + TN) / (TP + TN + FP + FN)
    return precision, recall, F1, acc

In [17]:
model = BertSERModel(bert_encoder=bert_encoder, pos_weight=pos_weight)
# model.to(device)
None

In [15]:
aa = next(iter(trainloader))
print(aa[0].shape)
print(aa[1].shape)
print(aa[2].shape)

torch.Size([64, 55])
torch.Size([64, 55])
torch.Size([64, 55])


In [18]:
model(next(iter(trainloader)))

RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
model.train()

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for batch in tqdm(trainloader):
        
        tokens_tensors, segments_tensors, masks_tensors, labels = [data.to(device) for data in batch]

        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        # grad clipping for preventing gradient explode(important!)
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        # losses in run time
        running_loss += loss.item()
        
    # calc accuracy
    # _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))

I0707 04:14:09.101490 140151046076224 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
